# Лабораторная работа №3
## Выполнил студент группы БФИ2001 Заморский Пётр Владимирович

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [1]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [2]:
def compare_chars(a, b, case_sensitive):
    if case_sensitive: return a == b
    return a.casefold() == b.casefold()

def string_prefix(string, case_sensitive = True):
    result = [0] * len(string)
    for i in range(1, len(string)):
        j = result[i - 1]
        while j > 0 and not compare_chars(string[j], string[i], case_sensitive):
            j = result[j - 1]
        if compare_chars(string[j], string[i], case_sensitive):
            j += 1
        result[i] = j
    return result

def kmp_find(source, target, only_first = False, case_sensitive = True, is_word = False):   
    result = []
    prefix = string_prefix(target)
    i = j = 0
    while i < len(source):
        while j > 0 and not compare_chars(source[i], target[j], case_sensitive):
            j = prefix[j - 1]
        if compare_chars(source[i], target[j], case_sensitive):
            j += 1
            if j == len(target):
                if not is_word or i == len(source) - 1 or not source[i + 1].isalnum():
                    if only_first: return i - j + 1
                    result.append(i - j + 1)
                j = prefix[j - 1]
        elif is_word and j == 0 and source[i].isalnum():
            i += 1
        i += 1
    return result

test_strings = ("AbcDABCDaabcdadbbabcABC" * 100 + "aaab" + "AbcDABCDaabcdadbbabcABC" * 100, "aaab")

start_time = t.now()

for _ in range(100):    
    kmp_find(test_strings[0], test_strings[1], True)

print(f"{(t.now() - start_time).microseconds} μs")

70999 μs


#### Упрощенный алгоритм Бойера-Мура

In [3]:
def compare_chars(a, b, case_sensitive):
    if case_sensitive: return a == b
    return a.casefold() == b.casefold()

def string_shift(string, case_sensitive = True):
    shift = defaultdict()
    last_char = string[-1]
    if not case_sensitive: last_char = last_char.casefold()
    for i in range(len(string) - 1):
        c = string[i]
        if not case_sensitive: c = c.casefold()
        if c != last_char:
            shift[c] = len(string) - i - 1
    return shift

def bm_find(source, target, only_first = False, case_sensitive = True, is_word = False):
    shift = string_shift(target, case_sensitive)
    i = len(target) - 1
    j = 0
    result = []
    while i < len(source):
        f = False
        if is_word and i < len(source) - 1 and source[i + 1].isalnum():
            c = source[i - j]
            if not case_sensitive: c = c.casefold()
            i += shift.get(c, len(target) - 1)
            j = 0
            continue
        while compare_chars(source[i - j], target[-(j + 1)], case_sensitive):
            j += 1
            if j == len(target):
                if not is_word or i - j + 1 == 0 or not source[i - j].isalnum():
                    if only_first: return i - j + 1
                    result.append(i - j + 1)
                    f = True
                    j = 0
                    i += len(target)
                break
        if not f:
            c = source[i - j]
            if not case_sensitive: c = c.casefold()
            i += shift.get(c, len(target) + j) - j
            j = 0
    return result

start_time = t.now()

for _ in range(100):    
    bm_find(test_strings[0], test_strings[1], True)

print(f"{(t.now() - start_time).microseconds} μs")

61998 μs


#### Стандартная функция

In [4]:
start_time = t.now()

for _ in range(100):    
    test_strings[0].find(test_strings[1])

print(f"{(t.now() - start_time).microseconds} μs")

1004 μs


#### Сравнение всех 3-х функций с пользовательским вводом

In [5]:
source = input("Source string: ")
target = input("Target substring to find: ")

start_time = t.now()
r = kmp_find(source, target)
print("KMP: ", r, ",  ", (t.now() - start_time).microseconds, " μs")
start_time = t.now()
r = bm_find(source, target)
print("BM: ", r, ",  ", (t.now() - start_time).microseconds, " μs")
start_time = t.now()
r = source.find(target)
print("Standart: ", r, ",  ", (t.now() - start_time).microseconds, " μs")


KMP:  [1, 7] ,   0  μs
BM:  [1, 7] ,   0  μs
Standart:  1 ,   0  μs


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [6]:
def check_15_valid(table):
    n = 0
    for i in range(1, 16):
        inx = table.index(i)
        row = (inx // 4) + 1
        count = 0
        for j in range(inx + 1, 16):
            v = table[j]
            if v > 0 and v < i: count += 1
        n += count + row 
    return n % 2 == 0

def cell_pos(index):
    return (index % 4, index // 4)

def cell_index(pos):
    return pos[0] * 4 + pos[1]

class AStarNode:
    def __init__(self, parent, step, distance, table, moved_num):
        self.parent = parent
        self.step = step
        self.distance = distance
        self.table = table
        self.space_index = table.index(0)
        self.space_position = cell_pos(self.space_index)
        self.moved_num = moved_num

    def __eq__(self, other):
        if isinstance(other, AStarNode):
            return self.table == other.table
        return False

    def __ne__(self, other):
        return not self.__eq__(other)
    
    @property
    def price(self):
        return self.step + self.distance * 10

    def get_neighbors(self):
        space_position = self.space_position
        result = []
        if space_position[0] > 0:
            result.append(self.get_neighbor((-1, 0)))
        if space_position[0] < 3:
            result.append(self.get_neighbor((1, 0)))
        if space_position[1] > 0:
            result.append(self.get_neighbor((0, -1)))
        if space_position[1] < 3:
            result.append(self.get_neighbor((0, 1)))
        return result
    
    def get_neighbor(self, move):
        new_table = self.table.copy()
        new_cell_index = cell_index((self.space_position[0] + move[0], self.space_position[1] + move[1]))
        moved_num = new_table[new_cell_index]
        new_table[self.space_index], new_table[new_cell_index] = moved_num, new_table[self.space_index]
        return AStarNode(self, self.step + 1, calc_distance(new_table), new_table, moved_num)

def calc_distance(table):
    result = 0
    for i in range(0, 16):
        if table[i] != i + 1 and table[i] > 0:
            result += 1
    return result

def resolve_15(table):
    if not check_15_valid(table): return []
    
    open = [AStarNode(None, 0, calc_distance(table), table, 0)]
    close = []

    def get_min_open():
        min_node = None
        min_price = 99999
        for n in open:
            if n.price < min_price:
                min_price = n.price
                min_node = n
        return min_node

    i = 0

    while len(open) > 0:
        current = get_min_open()
        #print(f"--- {i} ---")
        #print((current.step, current.distance))
        #print(current.table)

        i += 1

        if calc_distance(current.table) == 0:
            result = []
            while current is not None and current.moved_num > 0:
                result.append(current.moved_num)
                current = current.parent
            result.reverse()
            return result
        
        open.remove(current)
        close.append(current)
        neighbors = current.get_neighbors()
        for neighbor in neighbors:
            if neighbor in close: continue
            step = current.step + 1
            isBetter = False
            if neighbor not in open:
                neighbor.distance = calc_distance(neighbor.table)
                open.append(neighbor)
                isBetter = True
            else:
                isBetter = step < neighbor.step
            if isBetter:
                neighbor.parent = current
                neighbor.step = step

table = [2, 1, 3, 4, 6, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0]
resolve_15(table)

[12, 11, 7, 9, 1, 5, 6, 2, 5, 1, 9, 6, 2, 5, 1, 2, 6, 7, 11, 12]

### Вывод

Мы реализовали алгоритмы поиска подстроки в строке: алгоритм Кнута-Морриса-Пратта и упрощенный алгоритм Бойера-Мура (алгоритм Бойера-Мура-Хорспула), которые выполняют поиск за линейное время O(n).
Кроме того, реализован поиск решения игры в пятнашки через алгоритм A*.